In [2]:
!pip install some-new-package

Defaulting to user installation because normal site-packages is not writeable


ERROR: Could not find a version that satisfies the requirement some-new-package (from versions: none)

[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip
ERROR: No matching distribution found for some-new-package


In [5]:
!pip install faker

Defaulting to user installation because normal site-packages is not writeable
   ---------------------------------------- 0.0/2.0 MB ? eta -:--:--
   ---------- ----------------------------- 0.5/2.0 MB 3.3 MB/s eta 0:00:01
   --------------------- ------------------ 1.0/2.0 MB 3.0 MB/s eta 0:00:01
   ------------------------------- -------- 1.6/2.0 MB 3.0 MB/s eta 0:00:01
   ---------------------------------------- 2.0/2.0 MB 2.8 MB/s eta 0:00:00



[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


In [3]:
!python --version

Python 3.12.4


In [12]:
!pip install ipython-sql

Defaulting to user installation because normal site-packages is not writeable

   ---------------------------------------- 0/3 [ipython-genutils]
   ---------------------------------------- 0/3 [ipython-genutils]
   ------------- -------------------------- 1/3 [prettytable]
   -------------------------- ------------- 2/3 [ipython-sql]
   ---------------------------------------- 3/3 [ipython-sql]




[notice] A new release of pip is available: 25.1.1 -> 25.3
[notice] To update, run: python.exe -m pip install --upgrade pip


# 🏥 OPD & Doctor Performance Analytics  
## 📘 Notebook 1 - Data Generation

This notebook generates a **realistic synthetic healthcare dataset** for the OPD & Doctor Performance Analytics project.

### **Includes generation for:**
1. Branch  
2. Doctor  
3. OPD_Visit  
4. OPD_Diagnosis  
5. OPD_Prescription  
6. OPD_Billing  

### **Key Features**
- 80,000 OPD visits across 4 branches  
- Realistic timestamps, diagnoses, prescriptions  
- Proper relational structure  
- Suitable for SQL analytics tasks  
- Clean and optimized database with indexes  

Database created: **`hospital.db`**  


In [3]:
# Import Required Libraries

import random
from faker import Faker
import pandas as pd
import sqlite3
from datetime import datetime, timedelta
from tqdm import tqdm
import numpy as np

print("Libraries imported successfully.")


Libraries imported successfully.


In [4]:
# Initialize Data Generators

fake = Faker("en_IN")  # Indian names & cities
random.seed(42)
Faker.seed(42)

print("Faker initialized.")


Faker initialized.


In [5]:
# SQLite Database Initialization

DB_NAME = "hospital.db"
conn = sqlite3.connect(DB_NAME)

print("Connected to Database:", DB_NAME)


Connected to Database: hospital.db


## 🔧 Step 1 — Define Master Data  
These lists help generate realistic random values for:  
- Cities  
- Branch names  
- Specializations  
- Medicines  
- Diagnoses  
- Payment modes  


In [6]:
cities = ["Pune", "Mumbai", "Bengaluru", "Hyderabad"]
branch_names = ["Central Hospital", "City Care", "Green Valley", "Sunrise Clinic"]

specializations = [
    "General Physician", "Pediatrics", "Orthopedics",
    "Dermatology", "ENT", "Cardiology", "Neurology"
]

medicines = [
    "Paracetamol", "Amoxicillin", "Pantoprazole",
    "Cetirizine", "Metformin", "Atorvastatin", "Azithromycin"
]

diagnoses = [
    "Hypertension", "Diabetes", "Common Cold",
    "Back Pain", "Skin Infection", "Bronchitis", "Migraine"
]

payment_modes = ["Cash", "Card", "UPI", "Insurance"]

print("Master data defined.")


Master data defined.


## 🏢 Step 2 — Generate Branches  
We create 4 hospital branches with city mapping.  


In [7]:
print("Generating branches...")

branches = []
for i in range(4):
    branches.append({
        "branch_id": i + 1,
        "branch_name": branch_names[i],
        "city": cities[i]
    })

pd.DataFrame(branches).to_sql("Branch", conn, if_exists="replace", index=False)

print("Branches inserted:", len(branches))


Generating branches...
Branches inserted: 4


## 🩺 Step 3 — Generate Doctors  
- 10 doctors per branch  
- 40 doctors total  
- Specializations assigned randomly  


In [8]:
# 3. GENERATE DOCTORS (40 Total)

print("Generating doctors...")

doctors = []
doctor_id = 1

for branch in branches:
    for _ in range(10):   
        doctors.append({
            "doctor_id": doctor_id,
            "branch_id": branch["branch_id"],
            "doctor_name": fake.name(),
            "specialization": random.choice(specializations),
            "joining_date": fake.date_between(start_date='-10y', end_date='today').isoformat()
        })
        doctor_id += 1

pd.DataFrame(doctors).to_sql("Doctor", conn, if_exists="replace", index=False)

print("Doctors inserted:", len(doctors))


Generating doctors...
Doctors inserted: 40


## 🧾 Step 4 — Generate OPD Visits  
- 80,000 visits  
- Realistic timestamps & consultation types  
- Auto-generate diagnoses, prescriptions & billing  


In [9]:
# -------------------------------------------------------------
# 4. GENERATE OPD VISITS (80,000)
# -------------------------------------------------------------
print("Generating 80,000 OPD visits...")

total_visits = 80000
visits = []
visit_id = 1
patient_id_counter = 1
base_date = datetime(2023, 1, 1)

diagnosis_rows = []
prescription_rows = []
billing_rows = []

for _ in tqdm(range(total_visits), desc="Creating OPD Visits"):

    branch = random.choice(branches)
    assigned_doctor = random.choice([doc for doc in doctors if doc["branch_id"] == branch["branch_id"]])

    day_gap = random.randint(0, 1000)
    minute_gap = random.randint(8 * 60, 18 * 60)
    visit_datetime = base_date + timedelta(days=day_gap, minutes=minute_gap)

    consultation_type = random.choices(["New", "Follow-up"], [0.3, 0.7])[0]

    visits.append({
        "visit_id": visit_id,
        "patient_id": patient_id_counter,
        "doctor_id": assigned_doctor["doctor_id"],
        "branch_id": branch["branch_id"],
        "visit_datetime": visit_datetime.isoformat(sep=' '),
        "consultation_type": consultation_type
    })

    # Diagnosis
    for _ in range(random.choice([1, 1, 2])):
        diagnosis_rows.append({
            "diagnosis_id": None,
            "visit_id": visit_id,
            "diagnosis_name": random.choice(diagnoses)
        })

    # Prescriptions
    for _ in range(random.choice([0, 1, 2])):
        prescription_rows.append({
            "prescription_id": None,
            "visit_id": visit_id,
            "medicine_name": random.choice(medicines),
            "dose": f"{random.choice([1,2])} tablet(s) {random.choice(['OD','BD'])}",
            "duration_days": random.choice([3, 5, 7, 10])
        })

    # Billing
    consultation_fee = round(random.uniform(200, 1500), 2)
    additional_charges = round(random.uniform(0, 500), 2)
    discount_amount = random.choice([0, 0, 0, 50, 100])

    billing_rows.append({
        "bill_id": None,
        "visit_id": visit_id,
        "consultation_fee": consultation_fee,
        "additional_charges": additional_charges,
        "discount_amount": discount_amount,
        "payment_mode": random.choice(payment_modes)
    })

    visit_id += 1
    patient_id_counter += random.choices([0, 1], [0.7, 0.3])[0]

print("Visits generated:", len(visits))


Generating 80,000 OPD visits...


Creating OPD Visits: 100%|████████████████████████████████████████████████████| 80000/80000 [00:05<00:00, 14821.45it/s]

Visits generated: 80000


## 💾 Step 5 — Save All Tables to SQLite  


In [47]:
# -------------------------------------------------------------
# 5. STORE ALL DATA IN SQLITE
# -------------------------------------------------------------
print("Saving tables to SQLite...")

pd.DataFrame(visits).to_sql("OPD_Visit", conn, if_exists="replace", index=False)
pd.DataFrame(diagnosis_rows).to_sql("OPD_Diagnosis", conn, if_exists="replace", index=False)
pd.DataFrame(prescription_rows).to_sql("OPD_Prescription", conn, if_exists="replace", index=False)
pd.DataFrame(billing_rows).to_sql("OPD_Billing", conn, if_exists="replace", index=False)

print("All tables stored successfully!")


Saving tables to SQLite...
All tables stored successfully!


## ⚡ Step 6 — Create Indexes (Performance Optimization)  
Indexes improve SQL query execution speed.  


In [48]:
conn2 = sqlite3.connect("hospital.db")
cur = conn2.cursor()

indexes = [
    "CREATE INDEX IF NOT EXISTS idx_visit_datetime ON OPD_Visit(visit_datetime);",
    "CREATE INDEX IF NOT EXISTS idx_visit_doctor ON OPD_Visit(doctor_id);",
    "CREATE INDEX IF NOT EXISTS idx_visit_branch ON OPD_Visit(branch_id);",
    "CREATE INDEX IF NOT EXISTS idx_bill_visit ON OPD_Billing(visit_id);",
    "CREATE INDEX IF NOT EXISTS idx_bill_payment ON OPD_Billing(payment_mode);"
]

for idx in indexes:
    cur.execute(idx)

conn2.commit()
conn2.close()

print("Indexes created successfully!")


Indexes created successfully!


## 📊 Step 7 — Verify Inserted Row Counts  


In [49]:
conn = sqlite3.connect("hospital.db")

tables = ["Branch", "Doctor", "OPD_Visit", "OPD_Diagnosis", "OPD_Prescription", "OPD_Billing"]

for table in tables:
    count = conn.execute(f"SELECT COUNT(*) FROM {table}").fetchone()[0]
    print(f"{table}: {count} rows")

conn.close()


Branch: 4 rows
Doctor: 40 rows
OPD_Visit: 80000 rows
OPD_Diagnosis: 106733 rows
OPD_Prescription: 79827 rows
OPD_Billing: 80000 rows


# Data Generation Completed Successfully!
Your database **`hospital.db`** is ready for analytics.  
Proceed to **Notebook 2 -analysis.ipynb** to run SQL tasks.  
